In [61]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
import pandas as pd

from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [55]:
"""Read in data of interest"""

transactions = pd.read_csv("../prod_data/transactions.csv")
industry_descriptions = pd.read_csv('../enriched/industry_descriptions.csv')
sector_descriptions = pd.read_csv('../enriched/sector_descriptions.csv')
transactions = transactions.merge(industry_descriptions, on='industry', how='left').merge(sector_descriptions, on='sector', how='left')
transactions['string_object'] = (transactions['display_name'].astype(str)  + 's purchase of' +
                                 transactions['type'].astype(str)  + ' of ' + 
                                 transactions['asset_description'].astype(str)  + ' on ' + 
                                 transactions['combined_transaction_date'].astype(str) + ', ticker  ' + 
                                 transactions['ticker'].astype(str) + ' in the industry ' +
                                 transactions['industry'].astype(str)  + ' in the sector  ' +
                                 transactions['sector'].astype(str)  + ' which are described by ' +
                                 transactions['industry_description'].astype(str)  + ' and by ' +
                                 transactions['sector_description'].astype(str)  + 'in the' +
                                 transactions['congress'].astype(str) +"th Congress"
                                 
)

transactions

,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,ptr_link,amount_formatted,id,uuid,congress,industry_description,sector_description,string_object
0,Rep. Susie Lee,susie lee,2020-02-24,04/28/2020,BHC,Bausch Health Companies Inc.,purchase,"$1,001 - $15,000",Pharmaceuticals and Biotechnology,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,c1c5c603-f2ea-4c99-a9e4-5c158229c0e7,116,Companies involved in the production of pharma...,The Health Care sector consists of companies i...,Rep. Susie Lees purchase ofpurchase of Bausch ...
1,Rep. Susie Lee,susie lee,2020-03-06,04/28/2020,BAH,Booz Allen Hamilton Holding Corporation,sale_full,"$1,001 - $15,000",Professional Services,Consumer Services,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,b76332f3-3410-4ed0-82e6-ab19f484a330,116,Providers of various professional services.,The Consumer Services sector includes business...,Rep. Susie Lees purchase ofsale_full of Booz A...
2,Rep. Susie Lee,susie lee,2020-03-23,04/28/2020,BYD,Boyd Gaming Corporation,sale_full,"$1,001 - $15,000",Movies/Entertainment,Consumer Discretionary,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,37707207-7c2e-4ff3-9404-ceaeb052b1df,116,Businesses engaged in the entertainment and mo...,The Consumer Discretionary sector comprises co...,Rep. Susie Lees purchase ofsale_full of Boyd G...
3,Rep. Susie Lee,susie lee,2020-02-04,04/28/2020,CTLT,"Catalent, Inc.",purchase,"$1,001 - $15,000",Biotechnology: Pharmaceutical Preparations,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,1eaa4073-8742-4411-8df5-e27ce597a3e6,116,Biotechnology companies specializing in pharma...,The Health Care sector consists of companies i...,Rep. Susie Lees purchase ofpurchase of Catalen...
4,Rep. Susie Lee,susie lee,2020-03-18,04/28/2020,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Real Estate,Finance,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,60f1a149-4c5e-4f06-9ec6-d4c3189e99af,116,"Companies involved in real estate buying, sell...",The Finance sector includes companies involved...,Rep. Susie Lees purchase ofsale_partial of CBR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24121,Sen. Thomas Carper,thomas carper,2023-01-11,02/02/2023,FLNG,FLEX LNG Ltd. Ordinary Shares,Purchase,"$1,001 - $15,000",NaN,NaN,DE,https://efdsearch.senate.gov/search/view/ptr/d...,1000.0,C000174,8a87e854-6314-478f-addf-5ff12fb72859,118,Not applicable or not available,NaN,Sen. Thomas Carpers purchase ofPurchase of FLE...
24122,Sen. Thomas Carper,thomas carper,2023-01-11,02/02/2023,EQNR,Equinor ASA,Purchase,"$1,001 - $15,000",Integrated oil Companies,Energy,DE,https://efdsearch.senate.gov/search/view/ptr/d...,1000.0,C000174,cb62098f-922a-4b1e-bb6f-9725e1742b65,118,Integrated oil companies engaged in oil explor...,The Energy sector includes companies engaged i...,Sen. Thomas Carpers purchase ofPurchase of Equ...
24123,Sen. Thomas Carper,thomas carper,2023-01-11,02/02/2023,MCD,McDonald's Corporation Common Stock,Sale (Full),"$1,001 - $15,000",Restaurants,Consumer Services,DE,https://efdsearch.senate.gov/search/view/ptr/d...,1000.0,C000174,3037eb9c-70c2-4b79-99f3-7024bed1b246,118,Restaurants offering food and dining services.,The Consumer Services sector includes business...,Sen. Thomas Carpers purchase ofSale (Full) of ...
24124,Sen. Rick Scott,rick scott,2023-01-19,01/19/2023,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,NaN,NaN,NaN,https://efdsearch.senate.gov/search/view/ptr/6...,NaN,S001217,09100fe3-0102-4f22-8d2b-e65128ac6372,118,Not applicable or not available,NaN,Sen. Rick Scotts purchase ofnan of This filing...


In [56]:
committee_descriptions = pd.read_csv('../enriched/committees_enriched.csv')
committee_assignments = pd.read_csv('../prod_data/committee_assignments_of_interest.csv')
committee_assignments = committee_assignments.merge(committee_descriptions, on=['committee_id', 'congress', 'chamber'], how='left')
committee_assignments['string_object'] = (committee_assignments['member_name'].astype(str)  + 's service on the ' +
                                          committee_assignments['committee_name'].astype(str)  + ' in the ' +
                                          committee_assignments['chamber'].astype(str)  + ' in the ' +
                                          committee_assignments['congress'].astype(str)  + 'th Congress' + ". The committee is described as " +
                                          committee_assignments['committee_description'].astype(str)  + "ad its committee code is"+
                                          committee_assignments['committee_id'].astype(str)

                                          )

committee_assignments

,member_id,member_name,member_begin_date,congress,chamber,committee_id,committee_name,committee_description,string_object
0,M001198,Roger Marshall,2019-01-23,116,house,HSAG,Committee on Agriculture,The Committee on Agriculture is a standing com...,Roger Marshalls service on the Committee on Ag...
1,M001198,Roger Marshall,2019-01-23,116,house,HSSY,"Committee on Science, Space, and Technology","The Committee on Science, Space, and Technolog...",Roger Marshalls service on the Committee on Sc...
2,R000307,Pat Roberts,2019-01-09,116,senate,SSAF,"Committee on Agriculture, Nutrition, and Forestry","The Committee on Agriculture, Nutrition, and F...",Pat Robertss service on the Committee on Agric...
3,M000355,Mitch McConnell,2019-01-09,116,senate,SSAF,"Committee on Agriculture, Nutrition, and Forestry","The Committee on Agriculture, Nutrition, and F...",Mitch McConnells service on the Committee on A...
4,B001236,John Boozman,2019-01-09,116,senate,SSAF,"Committee on Agriculture, Nutrition, and Forestry","The Committee on Agriculture, Nutrition, and F...",John Boozmans service on the Committee on Agri...
...,...,...,...,...,...,...,...,...,...
1527,W000800,Peter Welch,NaN,118,joint,JSEC,Joint Economic Committee,The Joint Economic Committee (JEC) is a commit...,Peter Welchs service on the Joint Economic Com...
1528,S001199,Lloyd Smucker,NaN,118,joint,JSEC,Joint Economic Committee,The Joint Economic Committee (JEC) is a commit...,Lloyd Smuckers service on the Joint Economic C...
1529,M000317,Nicole Malliotakis,NaN,118,joint,JSEC,Joint Economic Committee,The Joint Economic Committee (JEC) is a commit...,Nicole Malliotakiss service on the Joint Econo...
1530,B001292,Donald Beyer,NaN,118,joint,JSEC,Joint Economic Committee,The Joint Economic Committee (JEC) is a commit...,Donald Beyers service on the Joint Economic Co...


In [57]:
subcommittee_descriptions = pd.read_csv('../enriched/subcommittees_enriched.csv')
subcommittee_assignments = pd.read_csv('../prod_data/subcommittee_assignments_of_interest.csv')
subcommittee_assignments = subcommittee_assignments.merge(subcommittee_descriptions, on=['subcommittee_id', 'congress', 'chamber'], how='left')
subcommittee_assignments

,member_id,member_name,member_begin_date,congress,chamber,committee_id,subcommittee_id,subcommittee_name,parent_committee_id,subcommittee_description
0,M001198,Roger Marshall,2019-02-06,116,house,HSAG,HSAG22,"Commodity Markets, Digital Assets, and Rural D...",HSAG,"The Commodity Markets, Digital Assets, and Rur..."
1,M001198,Roger Marshall,2019-02-06,116,house,HSAG,HSAG29,"Livestock, Dairy, and Poultry",HSAG,"The Livestock, Dairy, and Poultry subcommittee..."
2,M001198,Roger Marshall,NaN,116,house,HSSY,HSSY15,Research and Technology,HSSY,The Research and Technology subcommittee is a ...
3,M001198,Roger Marshall,NaN,116,house,HSSY,HSSY18,Environment,HSSY,The Environment subcommittee in the United Sta...
4,B001236,John Boozman,NaN,116,senate,SSAF,SSAF13,"Commodities, Risk Management, and Trade",SSAF,"The Commodities, Risk Management, and Trade su..."
...,...,...,...,...,...,...,...,...,...,...
2113,W000800,Peter Welch,NaN,118,senate,SSJU,SSJU01,"Competition Policy, Antitrust, and Consumer Ri...",SSJU,"The Competition Policy, Antitrust, and Consume..."
2114,W000800,Peter Welch,NaN,118,senate,SSJU,SSJU04,"Immigration, Citizenship, and Border Safety",SSJU,"The Immigration, Citizenship, and Border Safet..."
2115,W000800,Peter Welch,NaN,118,senate,SSJU,SSJU25,"Federal Courts, Oversight, Agency Action, and ...",SSJU,"The Federal Courts, Oversight, Agency Action, ..."
2116,W000800,Peter Welch,NaN,118,senate,SSJU,SSJU26,Intellectual Property,SSJU,The Intellectual Property subcommittee is a le...


In [58]:
test = transactions.head(20).copy()
test


,display_name,string_name,combined_transaction_date,disclosure_date,ticker,asset_description,type,amount,industry,sector,state,ptr_link,amount_formatted,id,uuid,congress,industry_description,sector_description,string_object
0,Rep. Susie Lee,susie lee,2020-02-24,04/28/2020,BHC,Bausch Health Companies Inc.,purchase,"$1,001 - $15,000",Pharmaceuticals and Biotechnology,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,c1c5c603-f2ea-4c99-a9e4-5c158229c0e7,116,Companies involved in the production of pharma...,The Health Care sector consists of companies i...,Rep. Susie Lees purchase ofpurchase of Bausch ...
1,Rep. Susie Lee,susie lee,2020-03-06,04/28/2020,BAH,Booz Allen Hamilton Holding Corporation,sale_full,"$1,001 - $15,000",Professional Services,Consumer Services,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,b76332f3-3410-4ed0-82e6-ab19f484a330,116,Providers of various professional services.,The Consumer Services sector includes business...,Rep. Susie Lees purchase ofsale_full of Booz A...
2,Rep. Susie Lee,susie lee,2020-03-23,04/28/2020,BYD,Boyd Gaming Corporation,sale_full,"$1,001 - $15,000",Movies/Entertainment,Consumer Discretionary,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,37707207-7c2e-4ff3-9404-ceaeb052b1df,116,Businesses engaged in the entertainment and mo...,The Consumer Discretionary sector comprises co...,Rep. Susie Lees purchase ofsale_full of Boyd G...
3,Rep. Susie Lee,susie lee,2020-02-04,04/28/2020,CTLT,"Catalent, Inc.",purchase,"$1,001 - $15,000",Biotechnology: Pharmaceutical Preparations,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,1eaa4073-8742-4411-8df5-e27ce597a3e6,116,Biotechnology companies specializing in pharma...,The Health Care sector consists of companies i...,Rep. Susie Lees purchase ofpurchase of Catalen...
4,Rep. Susie Lee,susie lee,2020-03-18,04/28/2020,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Real Estate,Finance,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,60f1a149-4c5e-4f06-9ec6-d4c3189e99af,116,"Companies involved in real estate buying, sell...",The Finance sector includes companies involved...,Rep. Susie Lees purchase ofsale_partial of CBR...
5,Rep. Susie Lee,susie lee,2020-03-24,04/28/2020,CNC,Centene Corporation,purchase,"$1,001 - $15,000",Medical Specialities,Health Care,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,9c50142d-ea90-48b2-a615-0442de3a8bdb,116,Medical practices specializing in specific med...,The Health Care sector consists of companies i...,Rep. Susie Lees purchase ofpurchase of Centene...
6,Rep. Susie Lee,susie lee,2020-03-31,04/28/2020,DG,Dollar General Corporation,purchase,"$1,001 - $15,000",Department/Specialty Retail Stores,Consumer Services,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,53908ea4-f679-4a3d-ac66-1d630debff91,116,Retail stores specializing in specific departm...,The Consumer Services sector includes business...,Rep. Susie Lees purchase ofpurchase of Dollar ...
7,Rep. Susie Lee,susie lee,2020-02-26,04/28/2020,DPZ,Domino's Pizza Inc,sale_partial,"$1,001 - $15,000",Food Distributors,Consumer Non-Durables,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,80b575d6-fa5d-4d1b-a7d3-e8e6857d0bc5,116,Companies involved in food distribution.,The Consumer Non-Durables sector encompasses c...,Rep. Susie Lees purchase ofsale_partial of Dom...
8,Rep. Susie Lee,susie lee,2020-02-05,04/28/2020,DNKN,"Dunkin' Brands Group, Inc.",sale_full,"$1,001 - $15,000",NaN,NaN,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,0eeb07f8-6037-4aca-b83a-f585450f4acc,116,Not applicable or not available,NaN,Rep. Susie Lees purchase ofsale_full of Dunkin...
9,Rep. Susie Lee,susie lee,2020-02-24,04/28/2020,ENV,"Envestnet, Inc",sale_full,"$1,001 - $15,000",EDP Services,Technology,NV,https://disclosures-clerk.house.gov/public_dis...,1000.0,L000590,61508c49-

In [59]:
def tokenize_text_data(text, tokenizer = False):
    tokenized_text = tokenizer(text, return_tensors="pt")['input_ids']
    return tokenized_text

model_name = "bert-base-uncased"  # You can change the model as needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_lambda = lambda x, y = tokenizer: tokenize_text_data(x, y)
col_tokenized = test['string_object'].apply(tokenizer_lambda)
col_tokenized

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to embed text using BERT
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)